Imports

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("OncoData") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,676 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [23.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,325 kB]
Get:13 https://ppa.launchpadcontent.net/ubunt

In [2]:
from pyspark.sql.window import Window


Create Dataframes from CSVs

In [3]:
import requests

path = "https://raw.githubusercontent.com/sarahrosegallagher/PySparkDemo/main/clinical_details.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'clinical_details.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

clinical_df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True)
clinical_df.show()

+----------+------------+--------------------+------------------+--------------+
|Patient_ID|Cancer_Stage|   Histological_Type|     Tumor_Size_cm|Grade_of_Tumor|
+----------+------------+--------------------+------------------+--------------+
|         1|   Stage III|            Lymphoma|  8.55040151862427|          High|
|         2|   Stage III|            Lymphoma| 8.743641564805078|           Low|
|         3|   Stage III|Squamous Cell Car...|3.2522622446427762|          High|
|         4|    Stage IV|            Lymphoma|1.3495126098648087|  Intermediate|
|         5|    Stage IV|Squamous Cell Car...|3.7293896320590054|           Low|
|         6|   Stage III|Squamous Cell Car...| 5.833741844769899|          High|
|         7|    Stage IV|Squamous Cell Car...| 3.939861176164368|          High|
|         8|     Stage I|Squamous Cell Car...| 8.450821034088298|  Intermediate|
|         9|     Stage I|Squamous Cell Car...| 3.443886242377677|           Low|
|        10|    Stage IV|   

In [4]:

path = "https://raw.githubusercontent.com/sarahrosegallagher/PySparkDemo/main/insurance_information.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'insurance_information.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

insurance_df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True)
insurance_df.show()

+----------+----------+--------------------+----------+-----+-------------+
|Patient_ID|      Date|       Provider_Name|Deductible|Copay|Coverage_Type|
+----------+----------+--------------------+----------+-----+-------------+
|         1|2022-03-12|    UnitedHealthcare|    1604.0| 32.0|          HMO|
|         1|2022-11-09|              Humana|     842.0| 21.0|          PPO|
|         1|2022-10-28|               Cigna|    1358.0| 43.0|         NULL|
|         1|2021-11-12|               Cigna|      NULL| 35.0|          EPO|
|         2|2021-06-19|                NULL|    2654.0| 23.0|         NULL|
|         2|2023-09-22|               Cigna|    1298.0| 50.0|          EPO|
|         3|2021-03-07|Blue Cross Blue S...|     818.0| 47.0|          HMO|
|         3|2021-10-14|               Aetna|    3071.0| NULL|          HMO|
|         3|2023-01-14|                NULL|    1941.0| NULL|          HMO|
|         4|2023-01-25|                NULL|     633.0| 20.0|          HMO|
|         4|

In [5]:
path = "https://raw.githubusercontent.com/sarahrosegallagher/PySparkDemo/main/patient_information.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'patient_information.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

patient_df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True)
patient_df.show()

+----------+-----------------+---+------+-------------+
|Patient_ID|             Name|Age|Gender|    Diagnosis|
+----------+-----------------+---+------+-------------+
|         1|  Tracy Hernandez| 56|Female|Breast Cancer|
|         2| Michelle Salazar| 69|Female|  Lung Cancer|
|         3|     John Roberts| 46|Female|Breast Cancer|
|         4|    Holly Parsons| 32|Female| Colon Cancer|
|         5| Mrs. Linda Smith| 60|  Male| Colon Cancer|
|         6|   Keith Faulkner| 25|Female|Breast Cancer|
|         7| Colleen Anderson| 78|  Male| Colon Cancer|
|         8|    Gregory Allen| 38|Female| Colon Cancer|
|         9| Barbara Williams| 56|Female|Breast Cancer|
|        10| Kenneth Williams| 75|  Male|Breast Cancer|
|        11|    Johnny Horton| 36|Female| Colon Cancer|
|        12|     Amber Thomas| 40|  Male| Colon Cancer|
|        13|   Andrew Vazquez| 28|Female| Colon Cancer|
|        14|   Jessica Murray| 28|Female|  Lung Cancer|
|        15|  Tamara Schaefer| 41|  Male|  Lung 

In [6]:
path = "https://raw.githubusercontent.com/sarahrosegallagher/PySparkDemo/main/treatment_information.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'treatment_information.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()

treatment_df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True)
treatment_df.show()

+----------+-----------------+--------------------+------------------+
|Patient_ID|   Treatment_Type|Treatment_Start_Date|            Dosage|
+----------+-----------------+--------------------+------------------+
|         1|Radiation Therapy|          2023-10-23|174.37463310597718|
|         2|     Chemotherapy|          2022-09-27|135.36222037486215|
|         3|Radiation Therapy|          2021-11-26|  59.5267744385002|
|         4|          Surgery|          2023-07-10| 55.52328014539871|
|         5|          Surgery|          2023-08-09| 70.07781782009498|
|         6|     Chemotherapy|          2021-10-27|52.050794724049595|
|         7|     Chemotherapy|          2022-04-26| 61.30385905286935|
|         8|     Chemotherapy|          2022-05-17| 153.7571595753368|
|         9|          Surgery|          2020-10-31|130.15194125441946|
|        10|Radiation Therapy|          2021-06-25|162.48661242049576|
|        11|          Surgery|          2020-06-27|186.97486283864643|
|     

Inspecting and Curating Data

In [7]:
def check_nulls(df: DataFrame):
  for column in df.columns:
    null_records_count = df.filter(F.col(column).isNull()).count()

    if null_records_count > 0:
      print(f"Column '{column}' has {null_records_count} null records")
    elif null_records_count == 0:
      print(f"Column {column} has no null records")


In [8]:
insurance_df.printSchema()

root
 |-- Patient_ID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Provider_Name: string (nullable = true)
 |-- Deductible: double (nullable = true)
 |-- Copay: double (nullable = true)
 |-- Coverage_Type: string (nullable = true)



In [9]:
check_nulls(insurance_df)

Column Patient_ID has no null records
Column Date has no null records
Column 'Provider_Name' has 57 null records
Column 'Deductible' has 68 null records
Column 'Copay' has 52 null records
Column 'Coverage_Type' has 67 null records


In [10]:
#identify instances where provider name is null
null_insurance = insurance_df\
.where((F.col('Provider_Name').isNull()))\
.select('Patient_ID','Date','Provider_Name')

null_insurance.show()

+----------+----------+-------------+
|Patient_ID|      Date|Provider_Name|
+----------+----------+-------------+
|         2|2021-06-19|         NULL|
|         3|2023-01-14|         NULL|
|         4|2023-01-25|         NULL|
|         5|2022-07-30|         NULL|
|         5|2022-09-23|         NULL|
|         8|2021-07-22|         NULL|
|         9|2022-10-28|         NULL|
|        10|2021-03-29|         NULL|
|        10|2022-04-01|         NULL|
|        10|2021-07-18|         NULL|
|        12|2022-04-02|         NULL|
|        13|2021-08-27|         NULL|
|        18|2022-09-21|         NULL|
|        18|2023-11-07|         NULL|
|        20|2021-08-05|         NULL|
|        21|2021-08-08|         NULL|
|        22|2023-08-21|         NULL|
|        26|2022-02-24|         NULL|
|        29|2023-10-07|         NULL|
|        30|2021-08-19|         NULL|
+----------+----------+-------------+
only showing top 20 rows



In [33]:
window_spec = Window.partitionBy("Patient_ID").orderBy("Date").rowsBetween(Window.unboundedPreceding, 0)

insurance_provider_filled = insurance_df.withColumn("Provider_Name", F.last(F.col("Provider_Name"), True).over(window_spec))\
                                        .withColumn("Deductible", F.last(F.col("Deductible"), True).over(window_spec))\
                                        .withColumn("Copay", F.last(F.col("Copay"), True).over(window_spec))\
                                        .withColumn("Coverage_Type", F.last(F.col("Coverage_Type"), True).over(window_spec))

insurance_provider_filled.show()

+----------+----------+--------------------+----------+-----+-------------+
|Patient_ID|      Date|       Provider_Name|Deductible|Copay|Coverage_Type|
+----------+----------+--------------------+----------+-----+-------------+
|         1|2021-11-12|               Cigna|      NULL| 35.0|          EPO|
|         1|2022-03-12|    UnitedHealthcare|    1604.0| 32.0|          HMO|
|         1|2022-10-28|               Cigna|    1358.0| 43.0|          HMO|
|         1|2022-11-09|              Humana|     842.0| 21.0|          PPO|
|         2|2021-06-19|                NULL|    2654.0| 23.0|         NULL|
|         2|2023-09-22|               Cigna|    1298.0| 50.0|          EPO|
|         3|2021-03-07|Blue Cross Blue S...|     818.0| 47.0|          HMO|
|         3|2021-10-14|               Aetna|    3071.0| 47.0|          HMO|
|         3|2023-01-14|               Aetna|    1941.0| 47.0|          HMO|
|         4|2022-07-16|    UnitedHealthcare|    3404.0| 41.0|          PPO|
|         4|

In [35]:
#result - nulls are reduced by about half (did not catch cases where no prior info)
check_nulls(insurance_provider_filled)

Column Patient_ID has no null records
Column Date has no null records
Column 'Provider_Name' has 20 null records
Column 'Deductible' has 21 null records
Column 'Copay' has 17 null records
Column 'Coverage_Type' has 25 null records


Investigate & Explore Data

In [ ]:
patient_df.printSchema()

root
 |-- Patient_ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Diagnosis: string (nullable = true)



In [ ]:
check_nulls(patient_df)

Column Patient_ID has no null records
Column Name has no null records
Column Age has no null records
Column Gender has no null records
Column Diagnosis has no null records


In [ ]:
check_nulls(treatment_df)

Column Patient_ID has no null records
Column Name has no null records
Column Age has no null records
Column Gender has no null records
Column Diagnosis has no null records


In [ ]:
check_nulls(clinical_df)

Column Patient_ID has no null records
Column Name has no null records
Column Age has no null records
Column Gender has no null records
Column Diagnosis has no null records


In [ ]:
print(patient_df.count())
print(clinical_df.count())
print(treatment_df.count())

100
100
100


In [ ]:
patient_df.summary().show()

+-------+------------------+------------+------------------+------+-------------+
|summary|        Patient_ID|        Name|               Age|Gender|    Diagnosis|
+-------+------------------+------------+------------------+------+-------------+
|  count|               100|         100|               100|   100|          100|
|   mean|              50.5|        NULL|             50.27|  NULL|         NULL|
| stddev|29.011491975882016|        NULL|19.176403323992865|  NULL|         NULL|
|    min|                 1|Aaron Wilson|                19|Female|Breast Cancer|
|    25%|                25|        NULL|                34|  NULL|         NULL|
|    50%|                50|        NULL|                51|  NULL|         NULL|
|    75%|                75|        NULL|                68|  NULL|         NULL|
|    max|               100| Wayne Smith|                79|  Male|  Lung Cancer|
+-------+------------------+------------+------------------+------+-------------+



In [ ]:
clinical_df.summary().show()

+-------+------------------+------------+--------------------+------------------+--------------+
|summary|        Patient_ID|Cancer_Stage|   Histological_Type|     Tumor_Size_cm|Grade_of_Tumor|
+-------+------------------+------------+--------------------+------------------+--------------+
|  count|               100|         100|                 100|               100|           100|
|   mean|              50.5|        NULL|                NULL| 5.392475494287356|          NULL|
| stddev|29.011491975882016|        NULL|                NULL|2.6552666350625533|          NULL|
|    min|                 1|     Stage I|      Adenocarcinoma|1.1377408626134629|          High|
|    25%|                25|        NULL|                NULL| 3.158086232416629|          NULL|
|    50%|                50|        NULL|                NULL| 5.115386454523557|          NULL|
|    75%|                75|        NULL|                NULL| 7.431355937619569|          NULL|
|    max|               100|  

In [ ]:
treatment_df.summary().show()

+-------+------------------+--------------+------------------+
|summary|        Patient_ID|Treatment_Type|            Dosage|
+-------+------------------+--------------+------------------+
|  count|               100|           100|               100|
|   mean|              50.5|          NULL|124.29488646391448|
| stddev|29.011491975882016|          NULL|  41.9404369348223|
|    min|                 1|  Chemotherapy| 50.38925365754699|
|    25%|                25|          NULL|  90.3008723037961|
|    50%|                50|          NULL|128.54034140225167|
|    75%|                75|          NULL|158.91812661077722|
|    max|               100|       Surgery|197.93854938002227|
+-------+------------------+--------------+------------------+



In [ ]:
diagnosis_value_count = patient_df.groupBy("Diagnosis").count()

total_count = patient_df.count()

percentage_column = diagnosis_value_count.withColumn("Diagnosis_Percentage", (F.col("count")/total_count) * 100)

In [ ]:
percentage_column.show()

+-------------+-----+--------------------+
|    Diagnosis|count|Diagnosis_Percentage|
+-------------+-----+--------------------+
|  Lung Cancer|   32|                32.0|
| Colon Cancer|   33|                33.0|
|Breast Cancer|   35|                35.0|
+-------------+-----+--------------------+



Joining and Filtering

In [ ]:
#Breast Cancer Histology, Stage, and Treatment

bc_patient_df = patient_df.filter(patient_df.Diagnosis == 'Breast Cancer')

bc_df = bc_patient_df.join(clinical_df, on="Patient_ID").join(treatment_df, on = "Patient_ID")

bc_df.show()

+----------+-----------------+---+------+-------------+------------+--------------------+------------------+--------------+-----------------+--------------------+------------------+
|Patient_ID|             Name|Age|Gender|    Diagnosis|Cancer_Stage|   Histological_Type|     Tumor_Size_cm|Grade_of_Tumor|   Treatment_Type|Treatment_Start_Date|            Dosage|
+----------+-----------------+---+------+-------------+------------+--------------------+------------------+--------------+-----------------+--------------------+------------------+
|         1|  Tracy Hernandez| 56|Female|Breast Cancer|   Stage III|            Lymphoma|  8.55040151862427|          High|Radiation Therapy|          2023-10-23|174.37463310597718|
|         3|     John Roberts| 46|Female|Breast Cancer|   Stage III|Squamous Cell Car...|3.2522622446427762|          High|Radiation Therapy|          2021-11-26|  59.5267744385002|
|         6|   Keith Faulkner| 25|Female|Breast Cancer|   Stage III|Squamous Cell Car...| 

In [ ]:
bc_stage_percentage = bc_df.groupBy("Cancer_Stage").agg(F.count("Patient_ID").alias("Total"),F.round((F.count("Patient_ID") / bc_df.count()) * 100, 2).alias("Percentage"))

In [ ]:
def calculate_stage_percentage(df: DataFrame, stage_column: str):
  total_count = df.count()

  stage_percentage_df = df.groupBy(stage_column).agg(F.count("Patient_ID").alias("Total"),F.round((F.count("Patient_ID")/total_count)*100,2).alias("Percentage"))

  return stage_percentage_df

In [ ]:
bc_stage_percentage_df = calculate_stage_percentage(bc_df, "Cancer_Stage")

bc_stage_percentage_df.show()

+------------+-----+----------+
|Cancer_Stage|Total|Percentage|
+------------+-----+----------+
|   Stage III|    7|      20.0|
|    Stage IV|   14|      40.0|
|     Stage I|    9|     25.71|
|    Stage II|    5|     14.29|
+------------+-----+----------+



In [ ]:
bc_histology_percentage_df = calculate_stage_percentage(bc_df,"Histological_Type")

bc_histology_percentage_df.show()

+--------------------+-----+----------+
|   Histological_Type|Total|Percentage|
+--------------------+-----+----------+
|            Lymphoma|   13|     37.14|
|      Adenocarcinoma|    9|     25.71|
|Squamous Cell Car...|   13|     37.14|
+--------------------+-----+----------+

